In [31]:
import numpy as np
import pandas as pd
import torch

In [32]:
dataSet_list = ('FrcSub', 'Math1', 'Math2', 'ASSIST_0910', 'ASSIST_2017')
dataSetName=dataSet_list[4]
basedir = 'D:/python_practice/认知诊断/有失误率和猜测率'
if dataSetName == 'FrcSub':
    read_dir = basedir+'/data/frcSub/'
    save_dir = basedir+'/output/frcSub/'
elif dataSetName == 'Math1':
    read_dir = basedir+'/data/math1/'
    save_dir = basedir+'/output/math1/'
elif dataSetName == 'Math2':
    read_dir = basedir+'/data/math2/'
    save_dir = basedir+'/output/math2/'
elif dataSetName == 'ASSIST_0910':
    read_dir = basedir+'/data/a0910/'
    save_dir = basedir+'/output/a0910/'
elif dataSetName == 'ASSIST_2017':
    read_dir = basedir+'/data/a2017/'
    save_dir = basedir+'/output/a2017/'
else:
    print('Dataset does not exist!')
    exit(0)
print('数据集：', dataSetName)

train_data=pd.read_csv(read_dir+'train.csv').set_index('user_id')
item_data=pd.read_csv(read_dir+'item.csv')

W_=np.loadtxt(save_dir+'W_.txt')
D_=np.loadtxt(save_dir+'D_.txt')

数据集： ASSIST_2017


In [33]:
def get_Q(W_,item):
    Q = np.zeros(W_.shape, dtype='bool')
    for idx in item.index:
        item_id = item.loc[idx, 'item_id']
        know_list = item.loc[idx, 'knowledge_code'].replace(
            '[', '').replace(']', '').split(',')
        for know in know_list:
            Q[item_id-1, int(know)-1] = True
    return torch.tensor(Q, dtype=torch.float)

In [34]:
def diagnosis(W_,score, prob_idx):  # 前向传播
    W_ = torch.tensor(W_).float()
    X_i = torch.tensor(score).float().reshape(1, -1)
    W_i = torch.softmax(W_[prob_idx], dim=0)
    A_i = X_i @ W_i
    return A_i,W_i

In [35]:
def format_data(record):
    train = [[],[], []]  # 学生,习题，得分
    stu_list = set(record.index)

    for stu in stu_list:
        stu_item = record.loc[[stu], 'item_id'].values - 1
        stu_score = record.loc[[stu], 'score'].values

        train[0].append([stu-1]*len(stu_item))
        train[1].append(stu_item)
        train[2].append(stu_score)

    return train

In [36]:
train=format_data(train_data)
Q=get_Q(W_,item_data)

In [37]:
# 找出数据集中答题错误率比较高的学习者
error_ratio_list=np.zeros(len(train[2]))
for i,score in enumerate(train[2]):
    error_ratio_list[i]=1-sum(score)/len(score)
highError_stu_list=np.argwhere(error_ratio_list>0).reshape(-1)

In [38]:
len(highError_stu_list)

1678

In [39]:
correct_rate_list=[]
# 遍历错误率较高的学习者
for stu in highError_stu_list:
    stu_A,stu_W=diagnosis(W_,train[2][stu],train[1][stu])
    # 找到学习者的强知识点,大于0.6的就算强知识点
    strong_c=np.argwhere(stu_A.reshape(-1)>0.6).reshape(-1)
    if len(strong_c)>0:
        # 找到对这些知识点贡献最多的习题
        most_cont_prob=np.argmax(stu_W[:,strong_c],axis=0)
        # 统计学习者作答这些习题的正确率
        most_cont_score=torch.tensor(train[2][stu][most_cont_prob])
        correct_rate=(most_cont_score.sum()/most_cont_score.numel()).item()
        correct_rate_list.append(correct_rate)

In [40]:
print('数据集：', dataSetName)

# 支持度
np.mean(correct_rate_list)

数据集： ASSIST_2017


0.9632335551271304